In [1]:
import xarray as xr
from distributed import Client
Client()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64413,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:64424,Total threads: 2
Dashboard: http://127.0.0.1:64427/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:64416,


In [2]:
#Load in ERA5 model level zarr from Google cloud
#From here: https://github.com/google-research/arco-era5

ds = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/model-level-1h-0p25deg.zarr-v1',
    storage_options=dict(token='anon'),
    chunks={"time":1,"hybrid":-1}
)
ds

<xarray.Dataset>
Dimensions:                              (time: 1323648, hybrid: 137,
                                          latitude: 721, longitude: 1440)
Coordinates:
  * hybrid                               (hybrid) float32 1.0 2.0 ... 137.0
  * latitude                             (latitude) float32 90.0 89.75 ... -90.0
  * longitude                            (longitude) float32 0.0 0.25 ... 359.8
  * time                                 (time) datetime64[ns] 1900-01-01 ......
Data variables: (12/14)
    divergence                           (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    fraction_of_cloud_cover              (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    geopotential                         (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    ozone_mass_mixing_ratio              (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    specific_cloud_ice_water_content     (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    specific_cloud_liquid_water_content  (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    ...                                   ...
    specific_snow_water_content          (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    temperature                          (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    u_component_of_wind                  (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    v_component_of_wind                  (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    vertical_velocity                    (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
    vorticity                            (time, hybrid, latitude, longitude) float32 dask.array<chunksize=(1, 137, 721, 1440), meta=np.ndarray>
Attributes:
    valid_time_start:  1940-01-01
    last_updated:      2024-08-27 14:20:51.626245
    valid_time_stop:   2024-03-31

In [3]:
#Slice down a bit

ds_sliced = ds.sel(
    time=slice("2024-01-24 00:00","2024-01-31 23:00"),
    latitude=slice(-38,-44),
    longitude=slice(140,150),
    hybrid=slice(100,137)
)[["u_component_of_wind","v_component_of_wind","geopotential"]]

In [4]:
ds_sliced["u_component_of_wind"]

<xarray.DataArray 'u_component_of_wind' (time: 192, hybrid: 38, latitude: 25,
                                         longitude: 41)>
dask.array<getitem, shape=(192, 38, 25, 41), dtype=float32, chunksize=(1, 38, 25, 41), chunktype=numpy.ndarray>
Coordinates:
  * hybrid     (hybrid) float32 100.0 101.0 102.0 103.0 ... 135.0 136.0 137.0
  * latitude   (latitude) float32 -38.0 -38.25 -38.5 ... -43.5 -43.75 -44.0
  * longitude  (longitude) float32 140.0 140.2 140.5 140.8 ... 149.5 149.8 150.0
  * time       (time) datetime64[ns] 2024-01-24 ... 2024-01-31T23:00:00
Attributes:
    long_name:      U component of wind
    standard_name:  eastward_wind
    units:          m s**-1

In [7]:
ds_sliced.to_netcdf("/Users/andrewb1/OneDrive - The University of Melbourne/Research Fellow/data/era5_test.nc")